In [1]:
import pandas as pd
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import pickle

import tensorflow as tf
import keras
from keras.backend import clear_session
from keras.datasets import mnist
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import Activation
from keras.layers import Dropout
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.models import Sequential, load_model, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, matthews_corrcoef, roc_auc_score, confusion_matrix, f1_score

import optuna

import psycopg2
from config import *

from datetime import timedelta

import utm
import time

from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import class_weight
import optuna

/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/glabata/.conda/envs/grapevine/lib/pyt

In [2]:
conexion = psycopg2.connect(database=db_database, 
                                user=db_user, 
                                password=db_password, 
                                host=db_host, 
                                port=db_port)

In [3]:
fechaHasta='2023-01-01'
pathModelos = '/data/proyectos/GRAPEVINE/Models/fjlacueva/Phenology/ANN_ForcingVar_ALL'
query="select * from public.sabana_sin_rad where fecha <= '"+ fechaHasta + "';"
sabana = pd.read_sql_query(query, con=conexion).drop_duplicates()
sabana

,phenologystageid,variedad,codigo,longitude,latitude,altitud,doc_id,fecha,estacion,season,...,do_Borja,do_Calatayud,do_Cariñena,do_Somontano,variedad_CABERNET SAUVIGNON,variedad_CHARDONNAY,variedad_GARNACHA,variedad_MAZUELA,variedad_SYRACH,variedad_TEMPRANILLO
0,0.0,GARNACHA,50004A03200057,-1.284122,41.332883,686.0,Cariñena,2016-01-05,Z01,2015_2016,...,0,0,1,0,0,0,1,0,0,0
1,0.0,TEMPRANILLO,50004A02500051,-1.253609,41.357499,587.0,Cariñena,2016-01-05,Z01,2015_2016,...,0,0,1,0,0,0,0,0,0,1
2,0.0,GARNACHA,50073A03200052,-1.231152,41.350539,578.0,Cariñena,2016-01-05,Z01,2015_2016,...,0,0,1,0,0,0,1,0,0,0
3,0.0,MAZUELA,50201A00200124,-1.225159,41.293432,702.0,Cariñena,2016-01-05,Z01,2015_2016,...,0,0,1,0,0,0,0,1,0,0
4,0.0,GARNACHA,50004A03200057,-1.284122,41.332883,686.0,Cariñena,2016-01-06,Z01,2015_2016,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104630,0.0,GARNACHA,50037A01500059,-1.565603,41.150106,868.0,Calatayud,2022-12-29,Z18,2022_2023,...,0,1,0,0,0,0,1,0,0,0
104631,0.0,GARNACHA,50037A01500059,-1.565603,41.150106,868.0,Calatayud,2022-12-30,Z18,2022_2023,...,0,1,0,0,0,0,1,0,0,0
104632,0.0,GARNACHA,50037A01500059,-1.565603,41.150106,868.0,Calatayud,2022-12-31,Z18,2022_2023,...,0,1,0,0,0,0,1,0,0,0
104633,0.0,GARNACHA,50037A01500059,-1.565603,41.150106,868.0,Calatayud,2023-01-01,Z18,2022_2023,...,0,1,0,0,0,0,1,0,0,0


In [4]:
sabana['doc_id']=sabana.doc_id.fillna('New')

In [5]:
sabana.doc_id.value_counts()

Cariñena     68387
Borja        28918
Calatayud     7141
Somontano      189
Name: doc_id, dtype: int64

In [6]:
parcelas_train=[]
parcelas_test=[]
parcelas_vali=[]
for do in sabana.doc_id.unique():
    print(do)
    parcelas_do_train, parcelas_do_no=train_test_split(list(sabana[sabana.doc_id==do].codigo.unique()),
                                                       random_state=17,test_size=0.4)
    parcelas_train=parcelas_train+parcelas_do_train
    if len(parcelas_do_no)>1:
        parcelas_do_vali, parcelas_do_test=train_test_split(parcelas_do_no,test_size=0.5, random_state=17)
        parcelas_train=parcelas_train+parcelas_do_train
        parcelas_test=parcelas_test+parcelas_do_test
        parcelas_vali=parcelas_vali+parcelas_do_vali
    else:
        pascelas_test=parcelas_test+parcelas_do_no
        

Cariñena
Borja
Calatayud
Somontano


In [7]:
parcelas_vali

['50073A08400031',
 '50073A03500094',
 '50182A01500002',
 '50004A03400175',
 '50073A02000066',
 '50018A03200112',
 '50073A03400128',
 '50073A08900011',
 '50018A00900050',
 '50004A02500059',
 '50295A03900002',
 '50010A00700271',
 '50006A02600039',
 '50114A01800135',
 '50114A02300049',
 '50154A02400063',
 '50154A00500730',
 '50046A01700135',
 '50170A01000234',
 '50029A00800166']

In [8]:
conexion.close()

In [9]:
temp=sabana.copy()
datos_list=[]
for id_terr in temp.codigo.unique():
    datos_camp_terr=temp[temp['codigo']==id_terr]
    datos_camp_terr['next_fecha']=datos_camp_terr['fecha'].shift(-7)
    datos_camp_terr['next_pheno']=datos_camp_terr['phenologystageid'].shift(-7)
    datos_camp_terr=datos_camp_terr[(datos_camp_terr.next_fecha-datos_camp_terr.fecha).dt.days==7]
    datos_list.append(datos_camp_terr)
datos_total=pd.concat(datos_list).reset_index(drop=True)
datos_total

/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,phenologystageid,variedad,codigo,longitude,latitude,altitud,doc_id,fecha,estacion,season,...,do_Cariñena,do_Somontano,variedad_CABERNET SAUVIGNON,variedad_CHARDONNAY,variedad_GARNACHA,variedad_MAZUELA,variedad_SYRACH,variedad_TEMPRANILLO,next_fecha,next_pheno
0,0.0,GARNACHA,50004A03200057,-1.284122,41.332883,686.0,Cariñena,2016-01-05,Z01,2015_2016,...,1,0,0,0,1,0,0,0,2016-01-12,0.0
1,0.0,GARNACHA,50004A03200057,-1.284122,41.332883,686.0,Cariñena,2016-01-06,Z01,2015_2016,...,1,0,0,0,1,0,0,0,2016-01-13,0.0
2,0.0,GARNACHA,50004A03200057,-1.284122,41.332883,686.0,Cariñena,2016-01-07,Z01,2015_2016,...,1,0,0,0,1,0,0,0,2016-01-14,0.0
3,0.0,GARNACHA,50004A03200057,-1.284122,41.332883,686.0,Cariñena,2016-01-08,Z01,2015_2016,...,1,0,0,0,1,0,0,0,2016-01-15,0.0
4,0.0,GARNACHA,50004A03200057,-1.284122,41.332883,686.0,Cariñena,2016-01-09,Z01,2015_2016,...,1,0,0,0,1,0,0,0,2016-01-16,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80476,10.0,CHARDONNAY,22196A00100148,-0.101093,42.094222,481.0,Somontano,2020-08-16,gv05,2019_2020,...,0,1,0,1,0,0,0,0,2020-08-23,10.0
80477,10.0,CHARDONNAY,22196A00100148,-0.101093,42.094222,481.0,Somontano,2020-08-17,gv05,2019_2020,...,0,1,0,1,0,0,0,0,2020-08-24,10.0
80478,10.0,CHARDONNAY,22196A00100148,-0.101093,42.094222,481.0,Somontano,2020-08-18,gv05,2019_2020,...,0,1,0,1,0,0,0,0,2020-08-25,10.0
80479,10.0,CHARDONNAY,22196A00100148,-0.101093,42.094222,481.0,Somontano,2020-08-19,gv05,2019_2020,...,0,1,0,1,0,0,0,0,2020-08-26,10.0


In [10]:
datos_total.next_pheno.value_counts()

0.0     34914
10.0    17918
9.0      7542
3.0      6857
1.0      3790
2.0      2857
8.0      2520
7.0      1442
4.0      1096
5.0       785
6.0       760
Name: next_pheno, dtype: int64

In [11]:
np.argmax(pd.get_dummies(np.hstack([range(1,11),datos_total.next_pheno.values]))[10:].values,axis=1)+1

array([ 1,  1,  1, ..., 11, 11, 11])

In [12]:
class_weight.compute_class_weight('balanced', classes=np.asarray(range(0,11)), y=datos_total.next_pheno.values)

array([0.20955647, 1.93046294, 2.56088714, 1.06700518, 6.67559721,
       9.32032426, 9.62691388, 5.07382423, 2.90335498, 0.97009474,
       0.40832987])

In [13]:
def objective(trial,train,vali):
    variedades=[col for col in train.columns if 'variedad_' in col]
    dos=[col for col in train.columns if 'do_' in col]
    variables_basic=['phenologystageid','dia', 'min', 'max', 'mean', 'std', 'median','diff', 
                     'longitude', 'latitude', 'altitud',
                     # GB
                     'winkler_4.5_2_TbaseMax_Cumm',
                     'winkler_4.5_2_TbaseMax_Cumm 2_semanas_atras',
                     'winkler_4.5_2_Tbase_Cumm 1_semanas_adelante',
                     'precip__2__Cumm 2_semanas_atras',
                     'gdd_4.5_1_Tbase_sum_Cumm 1_semanas_adelante',
                     'chillingDD_7.0_2_Tbase_sum_Cumm 2_semanas_atras',
                     'precip__2__Cumm 1_semanas_adelante',
                     # RF
                     'winkler_4.5_t0_TbaseMax_Cumm 1_semanas_atras',
                     'winkler_4.5_t0_TbaseMax_Cumm',
                     'precip__1__Cumm',
                     'precip__1__Cumm 1_semanas_atras',
                     'chillingDD_7.0_1_Tbase_sum_Cumm',
                     'chillingDD_7.0_1_Tbase_sum_Cumm 1_semanas_atras'
                     
                    ]+variedades+dos
    medidas=variables_basic
    
    medidas.append('next_pheno')
    train=train[medidas].dropna()
    vali=vali[medidas].dropna()
    
    print(len(medidas))
    
    weights=class_weight.compute_class_weight('balanced', classes=np.asarray(range(0,11)), 
                                      y=np.hstack([range(0,11),train['next_pheno'].values]))


    X_train=train.drop(['next_pheno'], axis=1).values
    X_vali=vali.drop(['next_pheno'], axis=1).values
    Y_vali=vali['next_pheno'].values
    Y_train=pd.get_dummies(np.hstack([range(0,11),train['next_pheno'].values]))[11:].values

    clear_session()

    inputA = Input(shape=(X_train.shape[1],), name='Entrada')

    #capas=trial.suggest_int('Numero de capas',low=1,high=3)
    capas=trial.suggest_int('Numero de capas',low=2,high=18)
    
    y = Dense(int(trial.suggest_discrete_uniform('Neuronas capa 0',low=32,high=512,q=32)))(inputA)
    y = Activation(trial.suggest_categorical('Activacion capa 0',["selu","linear","tanh","softmax"]))(y)
    
    for i in range(capas):
        y = Dense(int(trial.suggest_discrete_uniform('Neuronas ' + str(capas) + ' capas capa ' + 
                                                     str(i),low=128,high=512,q=128)))(y)
        y = Activation(trial.suggest_categorical('Activacion ' + str(capas) + ' capas capa ' + 
                                                 str(i),["selu","linear","tanh","softmax"]))(y)
        
    for i in range(2,0,-1):
        potencia=trial.suggest_discrete_uniform('Potencia ' + str(capas) + ' capas capa ' + str(-i),low=1,high=6,q=1)
        y = Dense(int(2**potencia))(y)
        y = Activation(trial.suggest_categorical('Activacion ' + str(capas) + 
                                                 ' capas capa ' + str(-i),["selu","linear","tanh","softmax"]))(y)
        
    z = Dense(11, activation="softmax")(y)

    model = Model(inputs=inputA, outputs=z)
    
    callback = [EarlyStopping(monitor='val_loss',patience=17, min_delta=1.7,restore_best_weights=False),
            ModelCheckpoint(pathModelos + '/checkpointed_model.h5', 
                            monitor='val_loss', save_best_only=True)]
    
    learning_rate=trial.suggest_categorical('Learning rate',[10**-3, 10**-2, 10**-1])
     
    choiceval = trial.suggest_categorical('Optimizador',['sgd','adam','rmsprop'])
    if choiceval == 'adam':
        optim = keras.optimizers.Adam(lr=learning_rate)
    elif choiceval == 'rmsprop':
        optim = keras.optimizers.RMSprop(lr=learning_rate)
    else:
        optim = keras.optimizers.SGD(lr=learning_rate)

#     weights = class_weight.compute_class_weight('balanced', classes=np.asarray([0,1]), y=Y_train)
    
    model.save(pathModelos+'/checkpointed_model.h5')
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
    batch_power=trial.suggest_int('Batch power',low=1,high=10)
    model.fit(X_train, Y_train, epochs=1000, verbose=0, batch_size=2**batch_power, 
              callbacks=callback, validation_split=0.2, class_weight=weights)
    
    model=load_model(pathModelos+'/checkpointed_model.h5')
        
    probs=model.predict(X_vali)
    preds=np.argmax(probs,axis=1)+1
    mcc=matthews_corrcoef(Y_vali,preds)
    print("MCC: ",mcc)
    print("Accuracy: ", accuracy_score(Y_vali,preds))
    SCORE=mcc
    
    nombre=pathModelos+'/model-fenologia-'+str(SCORE)
     
    #if SCORE>0.65:    
    if SCORE>0.15:    
        model_json = model.to_json()
        with open(nombre+".json", "w") as json_file:
            json_file.write(model_json)
        model.save_weights(nombre+".h5")
        with open(nombre+".pkl", 'wb') as pfile:
            pickle.dump(medidas, pfile, protocol=pickle.HIGHEST_PROTOCOL)
    
    print('#'*100)
    
    return mcc

In [ ]:
random_state  = 17
direction     = 'maximize'
n_trials      = 5
n_jobs        = 1
timeout       = None
verbosity     = 0
trozos=1000



train=datos_total[datos_total.codigo.isin(parcelas_train)]
vali=datos_total[datos_total.codigo.isin(parcelas_vali)]
train=train.drop(columns=['codigo','fecha'])
print(len(train))
vali=vali.drop(columns=['codigo','fecha'])

study_name = f'Fenologia'
sampler    = optuna.samplers.TPESampler(seed=random_state)

FILE = pathModelos+f'/resumen_optuna-fenologia-nn-mcc-parcelas.csv'

study = optuna.create_study(study_name=study_name,direction=direction,sampler=sampler,
                            storage='sqlite:////'+pathModelos+'/optuna-aceituna-fenologia-nn-mcc-parcelas.db', 
                            load_if_exists=True)

for j in range(trozos):
    study.optimize(func=lambda trial: objective(trial,train,vali),n_trials=n_trials,timeout=timeout)

    result_df = study.trials_dataframe()
    result_df.to_csv(FILE)
    time.sleep(1)

[I 2023-10-15 17:12:28,174] Using an existing study with name 'Fenologia' instead of creating a new one.


48799
35
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



[I 2023-10-15 17:41:27,462] Trial 28 finished with value: 0.0 and parameters: {'Numero de capas': 11, 'Neuronas capa 0': 384.0, 'Activacion capa 0': 'linear', 'Neuronas 11 capas capa 0': 128.0, 'Activacion 11 capas capa 0': 'softmax', 'Neuronas 11 capas capa 1': 512.0, 'Activacion 11 capas capa 1': 'selu', 'Neuronas 11 capas capa 2': 512.0, 'Activacion 11 capas capa 2': 'softmax', 'Neuronas 11 capas capa 3': 128.0, 'Activacion 11 capas capa 3': 'softmax', 'Neuronas 11 capas capa 4': 512.0, 'Activacion 11 capas capa 4': 'softmax', 'Neuronas 11 capas capa 5': 512.0, 'Activacion 11 capas capa 5': 'softmax', 'Neuronas 11 capas capa 6': 128.0, 'Activacion 11 capas capa 6': 'linear', 'Neuronas 11 capas capa 7': 512.0, 'Activacion 11 capas capa 7': 'softmax', 'Neuronas 11 capas capa 8': 128.0, 'Activacion 11 capas capa 8': 'tanh', 'Neuronas 11 capas capa 9': 128.0, 'Activacion 11 capas capa 9': 'tanh', 'Neuronas 11 capas capa 10': 128.0, 'Activacion 11 capas capa 10': 'linear', 'Potencia 11 c

MCC:  0.0
Accuracy:  0.05562163296718674
####################################################################################################
35
MCC:  0.0
Accuracy:  0.05562163296718674
####################################################################################################


[I 2023-10-15 18:00:49,215] Trial 29 finished with value: 0.0 and parameters: {'Numero de capas': 3, 'Neuronas capa 0': 416.0, 'Activacion capa 0': 'linear', 'Neuronas 3 capas capa 0': 512.0, 'Activacion 3 capas capa 0': 'linear', 'Neuronas 3 capas capa 1': 128.0, 'Activacion 3 capas capa 1': 'tanh', 'Neuronas 3 capas capa 2': 128.0, 'Activacion 3 capas capa 2': 'tanh', 'Potencia 3 capas capa -2': 6.0, 'Activacion 3 capas capa -2': 'softmax', 'Potencia 3 capas capa -1': 1.0, 'Activacion 3 capas capa -1': 'tanh', 'Learning rate': 0.1, 'Optimizador': 'rmsprop', 'Batch power': 2}. Best is trial 1 with value: 0.0.


...

MCC:  0.0
Accuracy:  0.05562163296718674
####################################################################################################
35


[I 2023-10-15 21:34:18,356] Trial 41 finished with value: 0.0 and parameters: {'Numero de capas': 11, 'Neuronas capa 0': 288.0, 'Activacion capa 0': 'selu', 'Neuronas 11 capas capa 0': 512.0, 'Activacion 11 capas capa 0': 'selu', 'Neuronas 11 capas capa 1': 128.0, 'Activacion 11 capas capa 1': 'softmax', 'Neuronas 11 capas capa 2': 128.0, 'Activacion 11 capas capa 2': 'linear', 'Neuronas 11 capas capa 3': 128.0, 'Activacion 11 capas capa 3': 'tanh', 'Neuronas 11 capas capa 4': 128.0, 'Activacion 11 capas capa 4': 'tanh', 'Neuronas 11 capas capa 5': 128.0, 'Activacion 11 capas capa 5': 'selu', 'Neuronas 11 capas capa 6': 512.0, 'Activacion 11 capas capa 6': 'softmax', 'Neuronas 11 capas capa 7': 128.0, 'Activacion 11 capas capa 7': 'selu', 'Neuronas 11 capas capa 8': 128.0, 'Activacion 11 capas capa 8': 'softmax', 'Neuronas 11 capas capa 9': 128.0, 'Activacion 11 capas capa 9': 'selu', 'Neuronas 11 capas capa 10': 128.0, 'Activacion 11 capas capa 10': 'selu', 'Potencia 11 capas capa -2'

MCC:  0.0
Accuracy:  0.05562163296718674
####################################################################################################
35


[I 2023-10-15 21:54:01,823] Trial 42 finished with value: 0.0 and parameters: {'Numero de capas': 14, 'Neuronas capa 0': 160.0, 'Activacion capa 0': 'linear', 'Neuronas 14 capas capa 0': 128.0, 'Activacion 14 capas capa 0': 'selu', 'Neuronas 14 capas capa 1': 128.0, 'Activacion 14 capas capa 1': 'selu', 'Neuronas 14 capas capa 2': 128.0, 'Activacion 14 capas capa 2': 'linear', 'Neuronas 14 capas capa 3': 512.0, 'Activacion 14 capas capa 3': 'selu', 'Neuronas 14 capas capa 4': 512.0, 'Activacion 14 capas capa 4': 'tanh', 'Neuronas 14 capas capa 5': 128.0, 'Activacion 14 capas capa 5': 'tanh', 'Neuronas 14 capas capa 6': 512.0, 'Activacion 14 capas capa 6': 'linear', 'Neuronas 14 capas capa 7': 512.0, 'Activacion 14 capas capa 7': 'softmax', 'Neuronas 14 capas capa 8': 128.0, 'Activacion 14 capas capa 8': 'selu', 'Neuronas 14 capas capa 9': 128.0, 'Activacion 14 capas capa 9': 'softmax', 'Neuronas 14 capas capa 10': 512.0, 'Activacion 14 capas capa 10': 'linear', 'Neuronas 14 capas capa 

MCC:  0.0
Accuracy:  0.05562163296718674
####################################################################################################
35
